<a href="https://colab.research.google.com/github/versant2612/jnotebooks/blob/main/Sentiment_Analysis_Getting_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data Scraping

In [ ]:
#Install Google play scraper: https://github.com/JoMingyu/google-play-scraper
!pip install google_play_scraper

     |████████████████████████████████| 52 kB 1.1 MB/s 
  Created wheel for google-play-scraper: filename=google_play_scraper-1.0.2-py3-none-any.whl size=24393 sha256=c85a8853695c37e8aeca7d015b4af5545623cc729aab5d072e2f2bc27786b841
  Stored in directory: /root/.cache/pip/wheels/98/99/eb/bbb9d24a5c526980647efc10336eaaeffcf07749f581111128
Successfully built google-play-scraper


In [ ]:
import json
import pandas as pd
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter

from google_play_scraper import Sort, reviews, app

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

#### Top 10 Food and Drinks Apps in Brazil (09/21/2021)

1. iFodd ⇒ br.com.brainweb.ifood

2. Zé delivery ⇒ com.cerveceriamodelo.modelonow

3. McDonald's ⇒ com.mcdo.mcdonalds

4. Habibi's ⇒ habibs.alphacode.com.br

5. 99Food ⇒ com.xiaojukeji.didi.brazil.customer

6. Uber Eats ⇒ com.ubercab.eats

7. Rappi ⇒ com.grability.rappi

8. Burker King ⇒ burgerking.com.br.appandroid

9. Madero ⇒ br.com.Madero

10. Aiqfome ⇒ com.vanuatu.aiqfome

Source: https://www.appannie.com/

In [ ]:
apps_ids = ['br.com.brainweb.ifood', 'com.cerveceriamodelo.modelonow',

'com.mcdo.mcdonalds', 'habibs.alphacode.com.br',

'com.xiaojukeji.didi.brazil.customer',

'com.ubercab.eats', 'com.grability.rappi',

'burgerking.com.br.appandroid', 'br.com.Madero',

'com.vanuatu.aiqfome']

Scraping data for each app

In [ ]:
app_infos = []

for ap in tqdm(apps_ids):
    info = app(ap, lang='en', country='us')
    del info['comments']
    app_infos.append(info)

100%|██████████| 10/10 [00:02<00:00,  4.49it/s]


In [ ]:
app_infos_df = pd.DataFrame(app_infos)
app_infos_df.head()

,title,description,descriptionHTML,summary,summaryHTML,installs,minInstalls,score,ratings,reviews,histogram,price,free,currency,sale,saleTime,originalPrice,saleText,offersIAP,inAppProductPrice,size,androidVersion,androidVersionText,developer,developerId,developerEmail,developerWebsite,developerAddress,privacyPolicy,developerInternalID,genre,genreId,icon,headerImage,screenshots,video,videoImage,contentRating,contentRatingDescription,adSupported,containsAds,released,updated,version,recentChanges,recentChangesHTML,editorsChoice,similarApps,moreByDeveloper,appId,url
0,iFood Delivery de Comida,Baixe o app e peça sua comida no iFood. Aqui v...,Baixe o app e peça sua comida no iFood. Aqui v...,Delivery of grocery orders & restaurant food i...,Delivery of grocery orders &amp; restaurant fo...,"50,000,000+",50000000,4.765971,7372114,1167238,"[100055, 36104, 202501, 811749, 6221705]",0,True,USD,False,None,None,None,False,None,75M,5.0,5.0 and up,iFood Delivery de Comida e Mercado,iFood+Delivery+de+Comida+e+Mercado,ifood@ifood.com.br,http://www.ifood.com.br,None,https://institucional.ifood.com.br/abrindo-a-c...,7021917525281657200,Food & Drink,FOOD_AND_DRINK,https://play-lh.googleusercontent.com/1Y_VGOwY...,https://play-lh.googleusercontent.com/Faula1rT...,[https://play-lh.googleusercontent.com/mp4veJL...,None,None,Everyone,None,None,False,"Apr 25, 2012",1631556201,9.120.0,"Olá, fã de comida boa! Esta versão não traz gr...","Olá, fã de comida boa! Esta versão não traz gr...",False,"[com.mercadopago.wallet, com.wabi.customer, co...",[global.maplink.and_pru],br.com.brainweb.ifood,https://play.google.com/store/apps/details?id=...
1,Zé Delivery de Bebidas - Gelada a Preço de Mer...,Zé Delivery: o maior app de bebidas do país! \...,Zé Delivery: o maior app de bebidas do país! <...,Cold drinks delivery with fast delivery and lo...,Cold drinks delivery with fast delivery and lo...,"5,000,000+",5000000,4.879586,392640,186472,"[6596, 1135, 2916, 11655, 370338]",0,True,USD,False,None,None,None,False,None,60M,5.0,5.0 and up,Zé Delivery,Z%C3%A9+Delivery,atendimento@ze.delivery,https://ze.delivery/,"R. Dr. Renato Paes de Barros, 1017 - Itaim Bib...",https://ze.delivery/terms?isApp=true,8661611515991206213,Food & Drink,FOOD_AND_DRINK,https://play-lh.googleusercontent.com/GZJkM5S7...,https://play-lh.googleusercontent.com/E2PcXg8l...,[https://play-lh.googleusercontent.com/miavcPa...,https://www.youtube.com/embed/UWejHPyYi0Y?ps=p...,https://play-lh.googleusercontent.com/E2PcXg8l...,Everyone,None,None,False,"Aug 19, 2016",1629147236,21.32.1,"Alô, alô, tá chegando uma atualização bem quen...","Alô, alô, tá chegando uma atualização bem quen...",False,"[com.wabi.customer, com.cornershopapp.android,...",[com.zedelivery.deliveryman],com.cerveceriamodelo.modelonow,https://play.google.com/store/apps/details?id=...
2,McDonald's App - Latinoamérica,Enter the new McDonald's App and get exclusive...,Enter the new McDonald&#39;s App and get exclu...,"Enjoy esclusive discounts, promotions and coup...","Enjoy esclusive discounts, promotions and coup...","10,000,000+",10000000,4.548815,473060,150983,"[27511, 5112, 16486, 55082, 368869]",0,True,USD,False,None,None,None,False,None,53M,6.0,6.0 and up,Arcos Dorados,Arcos+Dorados,apps@br.mcd.com,http://www.mcdonalds.com.ar,None,https://api-discover-mcd.gigigoapps.com/app/te...,7424281334030416541,Food & Drink,FOOD_AND_DRINK,https://play-lh.googleusercontent.com/TCx8i7sv...,https://play-lh.googleusercontent.com/GEr93a5r...,[https://play-lh.googleusercontent.com/gLTfmrF...,None,None,Everyone,None,None,False,"Mar 27, 2017",1630913072,3.0.4,Information security improvements.\r\nPerforma...,Information security improvements.<br>Performa...,False,"[com.mcdonalds.app, com.wabi.customer, com.lit...",[br.com.app.gpuu1676426.gpuuddceac8a3bd1d549b5...,com.mcdo.mcdonalds,https://play.google.com/store/apps/details?id=...
3,Habib's,Novo App Habib’s \r\n+ Ágil \r\n+ Moderno \r\n...,Novo App Habib’s <br>+ Ágil <br>+ Moderno <br>...,Get to kn

#### Scraping App Reviews

We want:
* Balanced dataset — roughly the same number of reviews for each score (1–5)
* A representative sample of the reviews for each app


We can satisfy the first requirement by using the scraping package option to filter the review score. For the second, we’ll sort the reviews by their helpfulness, which are the reviews that Google Play thinks are most important.

In [ ]:
app_reviews = []

for ap in tqdm(apps_ids):
    for score in list(range(1, 6)):
        for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
            rvs, _ = reviews(
                ap,
                lang='pt',
                country='br',
                sort=sort_order,
                count= 200 if score == 3 else 100,
                filter_score_with=score
            )
            for r in rvs:
                r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
                r['appId'] = ap
            app_reviews.extend(rvs)

100%|██████████| 10/10 [00:51<00:00,  5.15s/it]


In [ ]:
len(app_reviews)

11584

Saving reviews in a CSV file

In [ ]:
app_reviews_df = pd.DataFrame(app_reviews)

In [ ]:
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,sortOrder,appId
0,gp:AOqpTOG7ludjKBDYoRdqsIotlf0HrMb1hlmX2TeSFuI...,Pâmela Ferreira,https://play-lh.googleusercontent.com/a-/AOh14...,"Baixei o app, porém não tenho um uso constante...",1,6,9.120.0,2021-09-20 16:20:46,None,NaT,most_relevant,br.com.brainweb.ifood
1,gp:AOqpTOENQ5qBeVTVATwodjFdhOGh3jt8k4m_SnIila8...,Hugo Farias,https://play-lh.googleusercontent.com/a-/AOh14...,"ATENDIMENTO LIXO,pode parecer um aplicativo at...",1,86,9.120.0,2021-09-19 23:59:20,None,NaT,most_relevant,br.com.brainweb.ifood
2,gp:AOqpTOE0qONngImjzbowKH1gbLOia-AshfDX6A3Kg-d...,Carlos Wilton,https://play-lh.googleusercontent.com/a-/AOh14...,A falta de opções para ajudar o cliente quando...,1,50,9.120.0,2021-09-20 00:46:46,None,NaT,most_relevant,br.com.brainweb.ifood
3,gp:AOqpTOFchLfH2xwfwc9lzOOIkmPQQKO6fMSRJPuPvwZ...,Luiz Ponzo,https://play-lh.googleusercontent.com/a-/AOh14...,Excelente app Me incomoda ver menos do que 5 e...,1,16,9.120.0,2021-09-17 16:09:35,None,NaT,most_relevant,br.com.brainweb.ifood
4,gp:AOqpTOE7VvhDZBFb88xCwMO1lR5VY-HU6OTtd6qEe-e...,Victoria Lopes,https://play-lh.googleusercontent.com/a-/AOh14...,"A plataforma tem grande alcance, porém diversa...",1,358,9.117.1,2021-09-03 01:53:47,None,NaT,most_relevant,br.com.brainweb.ifood


In [ ]:
app_reviews_df.to_csv('reviews.csv', index=None, header=True)